In [1]:
import pandas as pd
import numpy as np
import os as os
import math
import datetime as dt
import matplotlib.pyplot as plt
from pypfopt import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from sklearn.preprocessing import MinMaxScaler 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
#Global Variables
date_of_interest = '01-March-2023' #Demarkation point after which prediction starts, inputted by the user
RiskFreeRate = 0.0425 #Current long-term average of US treasury rate for 10 year yield

In [3]:
# Defining Functions

# To prevent breaking of optimizer for assets returning exp return less than market free rate 
def optimize_safe(df):
    try:
        optimize(df)
    except:
        return 'LowSharpeRatio', 'error'

# Function to calculate net value change over a time period
def value_change(dataframe, date_of_interest, period = 60):
    startdate_safe = startdate #convert_datetimetimestamp_to_pd_timestamp(startdate)
    value_1 = df.loc[startdate_safe] @ dataframe
    value_2 = df.loc[startdate_safe+datetime.timedelta(days = period)] @ dataframe
    return value_2 - value_1

def df_splitter(df):
    return [pd.DataFrame(df[col]) for col in df.columns]

# Function to iterate a NxM dataframe into M-1 dataframes of Nx2 each, where index is preserved onto all M-1 generated dataframes
'''
def column_iterator(df):
    all_columns = df.columns
    results_so_far = []
    for col in all_columns:
        results_so_far.append(df[col])
        yield df[col],results_so_far
#df is the dataframe you want to iterate over - setting up of iterator
iterator_obj = column_iterator(df)
'''

'\ndef column_iterator(df):\n    all_columns = df.columns\n    results_so_far = []\n    for col in all_columns:\n        results_so_far.append(df[col])\n        yield df[col],results_so_far\n#df is the dataframe you want to iterate over - setting up of iterator\niterator_obj = column_iterator(df)\n'

In [4]:
# Read in price data
df = pd.read_csv("stock_prices.csv", parse_dates=True, index_col="date")

In [5]:
# Portfolio Optimizer

def optimize(df_to_opt):
    # Calculate expected returns and sample covariance
    mu = expected_returns.mean_historical_return(df_to_opt)
    S = risk_models.sample_cov(df_to_opt)

    # Optimize for maximal Sharpe ratio
    ef = EfficientFrontier(mu, S)
    raw_weights = ef.max_sharpe()
    cleaned_weights = ef.clean_weights()
    ef.save_weights_to_file("weights.csv")  # saves to file
    print(cleaned_weights)
    opt_result = ef.portfolio_performance(verbose=True)
    opt_weights = pd.DataFrame(cleaned_weights, columns=cleaned_weights.keys(), index=[0])
    
    return opt_result, opt_weights

### Work in progress from here:

In [6]:
sector = pd.read_csv("sp500SectorInfo.csv")
sector = sector[["symbol","security","sector","sub-industry"]]

data = pd.read_csv("data.csv", parse_dates=['date'])
data.set_index('date', inplace=True)
data = data[['key', 'close']]
#data = pd.pivot_table(data, index='date', columns='key')
data.head(1000)

,key,close
date,,
2018-06-05,PXD,191.160
2018-06-05,GOOGL,57.551
2018-06-05,AMGN,181.730
2018-06-05,CI,170.830
2018-06-05,CME,170.460
...,...,...
2018-06-07,CRM,132.910
2018-06-07,AIZ,97.500
2018-06-07,V,133.840


In [7]:
'''
list = ['GOOGL', 'AAPL', 'META', 'AMZN', 'GE', 'AMD', 'WMT', 'BAC', 'GM', 'T', 'XOM', 'BBY', 'MA', 'PFE', 'JPM', 'SBUX', 'PXD', 'UPS', 'NKE', 'LVS', 'ROK', 'GL']
data = data[data['key'].isin(list)]
data.shape
'''

"\nlist = ['GOOGL', 'AAPL', 'META', 'AMZN', 'GE', 'AMD', 'WMT', 'BAC', 'GM', 'T', 'XOM', 'BBY', 'MA', 'PFE', 'JPM', 'SBUX', 'PXD', 'UPS', 'NKE', 'LVS', 'ROK', 'GL']\ndata = data[data['key'].isin(list)]\ndata.shape\n"

In [8]:
df = data

In [9]:
df = pd.pivot_table(df, index='date', columns='key')
df

close                                                            \
key              A    AAL     AAP      AAPL    ABBV     ABC     ABT     ACGL   
date                                                                           
2018-06-05   63.52  44.18  129.95   48.3275   99.24   84.79   63.03  27.0267   
2018-06-06   65.19  44.19  130.59   48.4950   99.98   87.15   63.54  27.2000   
2018-06-07   65.23  43.84  130.05   48.3650   99.32   86.60   63.18  27.2133   
2018-06-08   65.73  43.11  131.71   47.9250  100.43   86.09   63.27  28.0967   
2018-06-11   66.13  44.16  132.73   47.8075   99.77   87.28   63.13  27.8267   
...            ...    ...     ...       ...     ...     ...     ...      ...   
2023-05-30  117.73  14.62  112.20  177.3000  136.44  166.01  101.71  71.2000   
2023-05-31  115.67  14.78   72.89  177.2500  137.96  170.15  102.00  69.7000   
2023-06-01  116.26  14.70   68.03  180.0900  133.44  172.59  102.82  69.6700   
2023-06-02  118.22  14.87   67.56  180.9500  136.87  174.52  104.20  70.9000   
2023-06-05  118.35  14.81   65.59  179.5800  136.86  174.69  103.76  71.3000   

                            ...                                                \
key            ACN    ADBE  ...    WYNN    XEL     XOM   XRAY     XYL     YUM   
date                        ...                                                 
2018-06-05  159.92  252.70  ...  176.88  43.32   80.68  42.29   71.66   82.91   
2018-06-06  162.71  253.95  ...  179.51  42.35   82.06  43.01   72.62   83.61   
2018-06-07  160.61  247.51  ...  176.25  42.72   82.88  43.11   73.11   84.19   
2018-06-08  162.17  251.21  ...  178.12  42.69   83.60  42.96   73.27   83.18   
2018-06-11  162.75  251.00  ...  180.82  42.13   83.10  43.40   71.82   83.11   
...            ...     ...  ...     ...    ...     ...    ...     ...     ...   
2023-05-30  308.43  417.21  ...   99.51  64.15  104.04  36.46   98.59  127.79   
2023-05-31  305.92  417.79  ...   98.70  65.29  102.18  36.12  100.20  128.69   
2023-06-01  305.20  426.75  ...   96.48  63.55  103.36  37.66  102.13  132.25   
2023-06-02  311.39  436.37  ...  100.35  63.14  105.76  38.79  105.06  134.53   
2023-06-05  304.45  434.18  ...  100.57  63.95  105.29  38.92  106.16  134.17   

                                           
key            ZBH    ZBRA   ZION     ZTS  
date                                       
2018-06-05  113.14  156.78    NaN   84.76  
2018-06-06  114.18  158.98    NaN   85.47  
2018-06-07  113.67  158.17    NaN   85.66  
2018-06-08  114.50  160.66    NaN   86.78  
2018-06-11  114.21  158.78    NaN   87.06  
...            ...     ...    ...     ...  
2023-05-30  126.43  267.78  28.91  160.94  
2023-05-31  127.34  262.57  27.29  163.01  
2023-06-01  127.36  266.20  27.67  166.00  
2023-06-02  129.81  278.01  29.07  171.88  
2023-06-05  130.47  269.11  28.72  171.12  

[1259 rows x 503 columns]

In [10]:
df.to_csv('OutFile.csv')

In [11]:
df_original = pd.read_csv("OutFile.csv", index_col='date', parse_dates=['date'])
df=df_original
df

ValueError: Missing column provided to 'parse_dates': 'date'

In [ ]:
#k = df[df.index.year.isin([2022, 2023])]

In [ ]:
# to check the optimum timeframe for optimiser

optimize_vs_time = []

for i in range(1, 4, 1):
    days = i*90    #90 days to step every quarter for upto 5 years
    df_temp = df.tail(days)
    opt_result, opt_weight = optimize(df_temp)
    optimize_vs_time.append(opt_result)
    
del df_temp

In [ ]:
optimize_vs_time

In [ ]:
sum = df.sum(axis=1)
sum

#### SP500 Data

In [ ]:
sp500 = pd.read_csv("INDEX_US_S&P 5yr.csv", parse_dates=['date'])
sp500.set_index('date', inplace=True)
sp500 = sp500[['close']]
sp500 = sp500.astype(float)
sp500.sort_index(inplace=True)
sp500

In [ ]:
sp500_opt_result, sp500_opt_weight = optimize(sp500)

In [ ]:
mu = expected_returns.mean_historical_return(sp500)
S = risk_models.sample_cov(sp500)

# Optimize for maximal Sharpe ratio
ef = EfficientFrontier(mu, S)

In [ ]:
# to check the optimum timeframe for optimiser

timescale = []
sp500_return_vs_time = []
sp500_volatility_vs_time = []

for i in range(1, 21, 1):
    days = i*90    #90 days to step every quarter for upto 5 years
    df_temp = sp500.tail(days)
    result1 = expected_returns.mean_historical_return(df_temp)
    result2 = risk_models.sample_cov(df_temp)
    #opt_result, opt_weight = optimize(df_temp)
    sp500_return_vs_time.append(result1[0])
    sp500_volatility_vs_time.append(result2['close'][0])
    timescale.append(i)
    print((result2['close'][0]))
    
del df_temp

In [ ]:
sp500_return_vs_time

In [ ]:
timescale

In [ ]:
sp500_perf = pd.DataFrame(list(zip(timescale, sp500_return_vs_time, sp500_volatility_vs_time)))
sp500_perf.reset_index
sp500_perf.rename(columns={0: "TimePeriod", 1:"Return", 2:'Volatility'}, inplace=True)
sp500_perf.set_index("TimePeriod", inplace=True)
sp500_perf["Return"] = sp500_perf["Return"]
sp500_perf["Sharpe Ratio"] = (sp500_perf["Return"] - RiskFreeRate)/sp500_perf["Volatility"]
sp500_perf

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))

sp500_perf.plot(y = 'Return', color='g', ax = ax)
sp500_perf.plot(y = 'Volatility', color='r', ax = ax)
sp500_perf.plot(y = 'Sharpe Ratio', color='b', ax = ax, linestyle='dashed', linewidth=3, secondary_y=True)
plt.ylabel("Return and Volatility")
ax.right_ax.set_ylabel('Sharpe Ratio')
ax.set_ylabel('Return and Volatility')
plt.tight_layout()

'''
#ax2 = ax.twinx()
ax.plot(sp500_perf["Return"], color='g')
ax.plot(sp500_perf["Volatility"], color='r')
ax2.plot(sp500_perf["Sharpe Ratio"], color='b', linewidth = 3, label='Sharpe Ratio')
ax.set_ylabel("Return & Volatility")
ax2.set_ylabel("Sharpe Ratio")
ax.set_xlabel("Timeperiod (in quarters)")
plt.legend()
plt.tight_layout()
'''

In [ ]:
sp500_perf['Sharpe Ratio']

### LSTM from here:

In [ ]:
#temp = sp500[sp500.index.year.isin([2020, 2022, 2023])]
temp = sp500

In [ ]:
window_size = 100 #Size of rolling window to train on (in days)

values = temp.values

training_data_len = math.ceil(len(values)* 0.8)

scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(values.reshape(-1,1))

train_data = scaled_data[0: training_data_len, :]

x_train = []
y_train = []

for i in range(window_size, len(train_data)):
    x_train.append(train_data[i-window_size:i, 0])
    y_train.append(train_data[i, 0])
    
x_train, y_train = np.array(x_train), np.array(y_train)

x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

In [ ]:
test_data = scaled_data[training_data_len-window_size: , : ]
x_test = []
y_test = values[training_data_len:]

for i in range(window_size, len(test_data)):
  x_test.append(test_data[i-window_size:i, 0])

x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

In [ ]:
x_test.shape

In [ ]:
model = keras.Sequential()
model.add(layers.LSTM(60, return_sequences=False, activation='relu', input_shape=(x_train.shape[1], 1)))
#model.add(layers.LSTM(100, activation='relu', return_sequences=False))
#model.add(layers.Dense(25))
model.add(layers.Dense(1))
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(x_train, y_train, batch_size= 1, epochs=4 ,verbose=True)

In [ ]:
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)
rmse = np.sqrt(np.mean(predictions - y_test)**2)
rmse

In [ ]:
temp2 = sp500
train = temp2[:training_data_len]
validation = temp2[training_data_len:]
validation['Predictions'] = predictions
plt.figure(figsize=(16,8))
plt.title('LSTM Model Prediction')
plt.xlabel('Date')
plt.ylabel('Close Price USD ($)')
#plt.plot(train)
plt.plot(validation[['close', 'Predictions']])
plt.legend(['Train', 'Val', 'Predictions'], loc='lower right')
plt.savefig('1.png')
#plt.show()

In [ ]:
plt.savefig('1.png')

In [ ]:
z = df.iloc[:,:]
#z = z[z.index.year.isin([2022, 2023])]
z

In [ ]:
%%capture

filtered_assets = []

for col in df:
    temp = pd.DataFrame(df[col])
    try:
        optimize(temp)
    except:
        continue
    else:
        filtered_assets.append(col)

In [ ]:
len(filtered_assets)

In [ ]:
q = df[filtered_assets]

In [ ]:
sp500_opt_result, sp500_opt_weight = optimize(q)

In [ ]:
df_filtered = q

In [ ]:
g = pd.DataFrame(filtered_assets)

In [ ]:
j = pd.DataFrame(df.columns)
j.rename(columns = {0:'ticker'}, inplace = True)
sector_dist = pd.merge(j, sector, how="left", left_on="ticker", right_on="symbol")
del j
sector_dist.drop("symbol", axis=1, inplace=True)
a = pd.DataFrame(sector_dist['sector'].value_counts())

In [ ]:
j = pd.DataFrame(df_filtered.columns)
j.rename(columns = {0:'ticker'}, inplace = True)
filt_sector_dist = pd.merge(j, sector, how="left", left_on="ticker", right_on="symbol")
del j
filt_sector_dist.drop("symbol", axis=1, inplace=True)
b = filt_sector_dist['sector'].value_counts()

In [ ]:
a['Remaining Count'] = b
a["Remaining %"] = 100*(a["Remaining Count"]/a["sector"])
a.sort_values('Remaining %', ascending=False, inplace=True)
a = a.round(1)

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(data=[
    #go.Bar(name='SP500', x=a.index, y=a['sector']),
    #go.Bar(name='SP500 Filtered', x=a.index, y=a['Remaining Count']),
    go.Bar(name='Remaining %', x=a.index, y=a['Remaining %'], text=a['Remaining %'])
])
# Change the bar mode
#fig.update_layout(barmode='group')
fig.update_layout(legend_title_text = "High Performers")
fig.update_xaxes(title_text="Sector")
fig.update_yaxes(title_text="% Companies Remaining in Sector")
fig.show()

In [ ]:
j = pd.DataFrame(df.columns)
j.rename(columns = {0:'ticker'}, inplace = True)
industry_dist = pd.merge(j, sector, how="left", left_on="ticker", right_on="symbol")
del j #deleting temp dataframes
industry_dist.drop("symbol", axis=1, inplace=True)
industry_dist = industry_dist.loc[industry_dist['sector']=='Real Estate']
c = pd.DataFrame(industry_dist['sub-industry'].value_counts())

In [ ]:
j = pd.DataFrame(df_filtered.columns)
j.rename(columns = {0:'ticker'}, inplace = True)
filt_sector_dist = pd.merge(j, sector, how="left", left_on="ticker", right_on="symbol")
del j #deleting temp dataframe
filt_sector_dist.drop("symbol", axis=1, inplace=True)
filt_sector_dist = filt_sector_dist.loc[filt_sector_dist['sector'] == 'Real Estate']
d = filt_sector_dist['sub-industry'].value_counts()

In [ ]:
c['Remaining Count'] = d
c['Remaining Count'] = c['Remaining Count'].fillna(0)
c["Remaining %"] = 100*c["Remaining Count"]/c["sub-industry"]
c.sort_values('Remaining %', ascending=False, inplace=True)
c = c.round(1)

In [ ]:
fig = go.Figure(data=[
    #go.Bar(name='SP500', x=a.index, y=a['sector']),
    #go.Bar(name='SP500 Filtered', x=a.index, y=a['Remaining Count']),
    go.Bar(name='Remaining %', x=c.index, y=c['Remaining %'], text=c['Remaining %'])
])
# Change the bar mode
#fig.update_layout(barmode='group')
fig.update_layout(legend_title_text = "High Performers")
fig.update_xaxes(title_text="Sub-Industries in Real Estate Sector")
fig.update_yaxes(title_text="% Companies Remaining")
fig.show()

del a, b, c, d #deleting temp dataframes